In [5]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [6]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec

from langchain_community.chat_message_histories import SQLChatMessageHistory

chat = ChatOpenAI()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're an assistant who's good at {ability}. Respond in 20 words or fewer",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | chat | StrOutputParser()

In [ ]:
def base64_to_dt(ts):
  return datetime.datetime.fromtimestamp(int.from_bytes(base64.b64decode(ts), "big"))

In [4]:
from secrets import token_hex

chat_id = token_hex(16)
print(chat_id)

e529c6f657530dc4e26a31d784321b1e


In [ ]:
from base64 import urlsafe_b64encode
from base64 import urlsafe_b64decode
import time
from datetime import datetime

chat_id = urlsafe_b64encode("%d" % int.from_bytes(datetime.now()))

print(chat_id)

chat_id_decode = int(urlsafe_b64decode(chat_id))

decoded = datetime.fromtimestamp(chat_id_decode)
print(decoded)

In [7]:
store = {}

def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]

In [10]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    # lambda session_id: SQLChatMessageHistory(
    #     session_id=session_id, connection_string="sqlite:///sqlite.db"
    # ),
    input_messages_key="input",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name="Conversation ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [14]:
response = with_message_history.invoke(
    {"ability": "math", "input": "add 1 with previous output"},
    config={"configurable": {"user_id": "masud@example.com", "conversation_id": chat_id}},
)
# "user_id": "123", what is 2+2?

In [12]:
response

'2 + 2 = 4'

In [13]:
store

{('masud@example.com',
  'e529c6f657530dc4e26a31d784321b1e'): ChatMessageHistory(messages=[HumanMessage(content='what is 2+2?'), AIMessage(content='2 + 2 = 4')])}

In [15]:
store

{('masud@example.com',
  'e529c6f657530dc4e26a31d784321b1e'): ChatMessageHistory(messages=[HumanMessage(content='what is 2+2?'), AIMessage(content='2 + 2 = 4'), HumanMessage(content='add 1 with previous output'), AIMessage(content='4 + 1 = 5')])}

In [17]:
type(store)

dict

In [26]:
k = store.keys()

In [28]:
print(k)

dict_keys([('masud@example.com', 'e529c6f657530dc4e26a31d784321b1e')])


In [29]:
type(k)

dict_keys

In [30]:
key = list(store.keys())[0]

In [34]:
key

('masud@example.com', 'e529c6f657530dc4e26a31d784321b1e')

In [32]:
key[0]

'masud@example.com'

In [33]:
key[1]

'e529c6f657530dc4e26a31d784321b1e'